![](https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png)

# Seguimiento de experimentos con MLFlow 3.X

## Aprendizaje Automático Aplicado

### Maestría en Ciencia de Datos

#### **Julio Waissman**, 2026

[**Abrir en google Colab**](https://colab.research.google.com/github/mcd-unison/aaa-curso/blob/main/ejemplos/mlflow3-et.ipynb)


Vamos a repetir en esta libreta los ejemplos que ponen en la [documentación de MLFlow 3.X](https://mlflow.org/docs/latest/ml/), pero con un enfoque más didáctico, para entender mejor lo que está pasando.

Vamos al mismo tiempo a ver con un servidor local (si se ejecuta localmente la libreta (no colab)) y con el servidor de MLFlow en la nube. Para esto, vamos a usar el mismo código, pero con diferentes configuraciones de MLFlow.

Vamos primer a configurar MLFlow para que use un servidor local, y luego lo configuraremos para que use el servidor de MLFlow en la nube. Usa o uno u otro, pero no ambos al mismo tiempo, para evitar confusiones.

## Opción A: Servidor local

Para servidor local hay que instalar la librería de MLFlow primero:

In [ ]:
!pip install --upgrade "mlflow>=3.1"

Y ahora vamos a usar un servidor de seguimiento de experimentos de MLFlow, el cual puede der de alguna de estas 3 configuraciones:

![](https://mlflow.org/docs/latest/assets/images/tracking-setup-overview-3d8cfd511355d9379328d69573763331.png)

En [Esta parte de la documentación](https://mlflow.org/docs/latest/self-hosting/architecture/overview/) explican un poco más sobre cada una de estas configuraciones, nos vamos a quedar con la primera y con la configuración de base.

1. **Backend Store**: Es donde se guardan los metadatos de los experimentos, como los parámetros, métricas, artefactos, etc. Puede ser una base de datos SQL (como SQLite, MySQL, PostgreSQL) o un sistema de archivos local. Por defaul es un archivo sqlite `mlflow.db` que se guarda en el directorio `mlruns` del proyecto.
2.  **Artifact Store**: Es donde se guardan los artefactos de los experimentos, como los modelos entrenados, los gráficos, etc. Puede ser un sistema de archivos local o un servicio de almacenamiento en la nube (como Amazon S3, Azure Blob Storage, Google Cloud Storage). Por default es el mismo directorio `mlruns` del proyecto.
3.  **MLFlow Tracking Server**: Es el servidor que se encarga de recibir las solicitudes de los clientes (como la libreta de Jupyter) y de interactuar con el Backend Store y el Artifact Store para guardar los datos de los experimentos. Puede ser un servidor local o un servidor en la nube. Por default es un servidor local que se ejecuta en el puerto 5000.

Y esto lo vamos a hacer desde la consola de comandos, no desde la libreta de Jupyter, para que quede claro que es algo que se hace a nivel de sistema operativo y no a nivel de código Python. Asi que en tu consola de comandos, en el directorio donde quieras que se guarde tu información de los experimentos, ejecuta el siguiente comando para iniciar el servidor local de MLFlow:

```bash
mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns --port 5000
```

y desde la libreta de Jupyter, vamos a configurar MLFlow para que use este servidor local, lo cual se hace con el siguiente código:

In [ ]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("MLflow Quickstart")

## Opción B: Usando el servidor de MLFlow en la nube

Para esto vamos a usar nuestra cuenta gratuita de [Databricks free](https://login.databricks.com/signup?provider=DB_FREE_TIER&dbx_source=www&itm_data=dbx-web-get-started&itm_source=www&itm_category=try-databricks&itm_page=try-databricks&itm_location=body&itm_component=general-asset-card&itm_offer=free-edition&tuuid=04b66209-37ed-4256-b8b6-15c5c789fc24&intent=SIGN_UP&rl_aid=025cddc3-65ad-472b-bb95-75908c66b88e&sisu_state=eyJsZWdhbFRleHRTZWVuIjp7Ii9zaWdudXAiOnsidG9zIjp0cnVlLCJwcml2YWN5Ijp0cnVlLCJjb3Jwb3JhdGVFbWFpbFNoYXJpbmciOnRydWV9fX0%3D), que nos da acceso a un servidor de MLFlow en la nube, y vamos a configurar MLFlow para que use este servidor.

Lo primero es encontrar la API Key de nuestro servidor de MLFlow en la nube, lo cual se hace desde la interfaz web de Databricks, en la sección de "User Settings" y luego en la sección de "Access Tokens". Ahí vamos a generar un nuevo token y lo vamos a copiar.

Ahora nos vamos a la seccion *Experiments* del menu vertical de la derecha, en la sección *AI/ML*. Ahi vamos a seleccionar *Custom Model Trainning* y vamos a crear un experimento con el nombre "mi primer experimento en la nube". 

Si vas a hacerlo local, entonces lo mejor es usar la librería `python-dotenv` para guardar la URL del servidor de MLFlow y la API Key en un archivo `.env`, y luego cargar estas variables de entorno desde la libreta de Jupyter. Esto lo hacemos para no tener que escribir la URL y la API Key directamente en el código, y para no exponer esta información sensible en el código. Recuerda que si lo guardas en un repostorio de github, tus archivos *.env* deben estar en el *.gitignore* para no exponer claves. Las variables que se necesitan son las siguientes:

```bash
DATABRICKS_TOKEN=<databricks-personal-access-token>
DATABRICKS_HOST=https://<workspace-name>.cloud.databricks.com
MLFLOW_TRACKING_URI=databricks
MLFLOW_EXPERIMENT_ID=<experiment-id>
```

Si ya las tienes en un archivo de texto llamado `.env`, entonces lo que tienes que hacer es cargar estas variables de entorno desde la libreta de Jupyter, lo cual se hace con el siguiente código:

```python
from dotenv import load_dotenv
load_dotenv()
```

Si lo usas en Colab, lo mejor es guardar *`secrets`* para no exponer nuestra información. Para esto, en Colab, vamos a la sección de "Secrets" en el menú lateral izquierdo, y ahí vamos a crear un nuevo secreto con el nombre `DATABRICKS_TOKEN` y el valor de nuestra API Key, y otro secreto con el nombre `DATABRICKS_HOST` y el valor de la URL de nuestro servidor de MLFlow en la nube y así tambien para `MLFLOW_TRACKING_URI` y `MLFLOW_EXPERIMENT_ID`. Luego, para cargar estas variables de entorno desde la libreta de Jupyter, lo cual se hace con el siguiente código:

```python
from google.colab import userdata
import os

DATABRICKS_TOKEN = userdata.get('DATABRICKS_TOKEN')
DATABRICKS_HOST = userdata.get('DATABRICKS_HOST')
MLFLOW_TRACKING_URI = userdata.get('MLFLOW_TRACKING_URI')
MLFLOW_EXPERIMENT_ID = userdata.get('MLFLOW_EXPERIMENT_ID')

os.environ['DATABRICKS_TOKEN'] = DATABRICKS_TOKEN
os.environ['DATABRICKS_HOST'] = DATABRICKS_HOST
os.environ['MLFLOW_TRACKING_URI'] = MLFLOW_TRACKING_URI
os.environ['MLFLOW_EXPERIMENT_ID'] = MLFLOW_EXPERIMENT_ID
```

## Cargando las librerías necesarias

Lo que necesitamos para ejecutar un experimentito con el foco en el uso de experiment tracking de *MLFlow* es lo siguiente:

In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Estableciendo el proceso para entrenar un modelo

Vamos a separtar en conjunto de prueba y conjunto de entrenamiento, y vamos a usar un modelo de regresión logística para el conjunto de datos mas sobado del mundo.

In [ ]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "random_state": 8888,
}

# Usando autologging

Si el experimento ya existe, va a crear nuevas corridas en el mismo experimento, y si no existe, lo va a crear automáticamente. Y ahora vamos a usar la opción de autologging de MLFlow con el sabor de scikit-learn, lo cual se hace con el siguiente código:

In [ ]:
mlflow.sklearn.autolog()

## El aprendizaje
Y ahora vamos a entrenar el modelo, lo cual se hace con el siguiente código:

In [ ]:
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

Con solo iniciar el autologging y entrenar el modelo, MLFlow ya va a guardar toda la información del experimento, como los parámetros del modelo, las métricas de evaluación, los artefactos (como el modelo entrenado), etc. Y esto lo podemos ver desde la interfaz web de MLFlow, que se encuentra en `http://localhost:5000` si estamos usando el servidor local.

Vamos a dar una vuelta por la interfaz web de MLFlow para ver toda la información que se ha guardado del experimento, y luego vamos a ver como se cargan los parámetros manualmente.

## Guardando los parámetros y los artefactos manualmente

Se pueden combinar ambas cosas pero vamos a ver directamente como hacer esto manualmente, sin usar el autologging, para entender mejor lo que está pasando. Para esto, vamos a usar el siguiente código:

In [ ]:
# Start an MLflow run
mlflow.sklearn.autolog(disable=True)  # Disable autologging for this run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Train the model
    lr = LogisticRegression(**params)
    lr.fit(X_train, y_train)

    # Log the model
    model_info = mlflow.sklearn.log_model(sk_model=lr, name="iris_model")

    # Predict on the test set, compute and log the loss metric
    y_pred = lr.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    # Optional: Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")
    
print("Model trained and logged successfully!")
print(model_info)

y volvemos a dar una vuelta al *Experiment tracking* para ver que está pasando con los parámetros y los artefactos que hemos guardado manualmente.

## Cargando un modelo desde el *Experiment tracking*
Y ahora vamos a cargar el modelo que hemos guardado manualmente, para ejecutarlo, lo cual se hace con el siguiente código:

In [ ]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

Si hubieramos querido cargar el modelo como un modelo con el sabor de scikit-learn, lo cual se hace con el siguiente código:

```python
loaded_model = mlflow.sklearn.load_model(model_info.model_uri) 
````
Y esto lo veremos con calma más adelante.